In [1]:
%matplotlib notebook
import logging
import os
import math
import multiprocessing
import json
import re
from itertools import combinations 

from scipy.stats import t

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
results_dir = "/hdd/results-analysis/data/processed_data/mcs7"
configs = {
    "0.06vpm" : {
        "Analytical": "analytical-7mcs-006vpm-2020-01-01-00_00_00",
        "Simulated" : "MCS7-006vpm-20dbm-2020-08-17-16_22_02"
    },
    "0.09vpm" : {
        "Analytical": "analytical-7mcs-009vpm-2020-01-01-00_00_00",
        "Simulated" : "MCS7-009vpm-20dbm-2020-08-18-07_49_49"
    },
    "0.12vp-10dbm" : {
        "Analytical": "analytical-7mcs-012vpm-10dbm-2020-01-01-00_00_00",
        "Simulated" : "new-result-012vpm-10dbm-2020-08-13-11_58_56"
    },
    "0.12vpm-20dbm" : {
        "Analytical": "analytical-7mcs-012vpm-20dbm-2020-01-01-00_00_00",
        "Simulated" : "MCS7-012vpm-20dbm-2020-08-18-07_50_17"
    },
    "0.12vpm-23dbm" : {
        "Analytical": "analytical-7mcs-012vpm-23dbm-2020-01-01-00_00_00",
        "Simulated" : "new-result-012vpm-23dbm-2020-08-13-11_59_30"
    },
    "0.16vpm" : {
        "Analytical": "analytical-7mcs-016vpm-2020-01-01-00_00_00",
        "Simulated" : "new-result-016vpm-2020-08-13-11_59_30"
    },
    "0.2vpm": {
        "Analytical": "analytical-7mcs-02vpm-2020-01-01-00_00_00",
        "Simulated" : "MCS7-02vpm-20dbm-2020-08-17-16_22_02"
    },
    "0.25vpm" : {
        "Analytical": "analytical-7mcs-025vpm-2020-01-01-00_00_00",
        "Simulated" :  "new-result-025vpm-2020-08-14-11_48_48"
    },
    "0.3vpm" : {
        "Analytical": "analytical-7mcs-03vpm-2020-01-01-00_00_00",
        "Simulated" : "MCS7-03vpm-20dbm-2020-08-17-16_20_01"
    }
}

In [3]:
config = "0.06vpm"
pdr_df = pd.read_csv("{}/{}/PDR-TB.csv".format(results_dir, configs[config]["Analytical"]))
pdr = pdr_df["Mean"].to_list()

In [21]:
print("="*47)
print("|Config       | PDR |  HD |  SEN |  PRO |  COL |")
print("="*47)

for config in configs:
    # Load analytical pdr
    a_pdr_df = pd.read_csv("{}/{}/PDR-TB.csv".format(results_dir, configs[config]["Analytical"]))
    a_pdr = a_pdr_df["Mean"].to_list()

    # Load simulated pdr
    s_pdr_df = pd.read_csv("{}/{}/PDR-TB.csv".format(results_dir, configs[config]["Simulated"]))
    s_pdr = s_pdr_df["Mean"].to_list()

    # Load analytical hd
    a_hd_df = pd.read_csv("{}/{}/hd_errors.csv".format(results_dir, configs[config]["Analytical"]))
    a_hd = a_hd_df["Mean"].to_list()

    # Load simulated hd
    s_hd_df = pd.read_csv("{}/{}/hd_errors.csv".format(results_dir, configs[config]["Simulated"]))
    s_hd = s_hd_df["Mean"].to_list()

    # Load analytical unsensed
    a_uns_df = pd.read_csv("{}/{}/unsensed_errors.csv".format(results_dir, configs[config]["Analytical"]))
    a_uns = a_uns_df["Mean"].to_list()

    # Load simulated unsensed
    s_uns_df = pd.read_csv("{}/{}/unsensed_errors.csv".format(results_dir, configs[config]["Simulated"]))
    s_uns = s_uns_df["Mean"].to_list()

    # Load analytical porpagation
    a_prop_df = pd.read_csv("{}/{}/prop_errors.csv".format(results_dir, configs[config]["Analytical"]))
    a_prop = a_prop_df["Mean"].to_list()

    # Load simulated propagation
    s_prop_df = pd.read_csv("{}/{}/prop_errors.csv".format(results_dir, configs[config]["Simulated"]))
    s_prop = s_prop_df["Mean"].to_list()

    # Load analytical collsiion
    a_int_df = pd.read_csv("{}/{}/interference_errors.csv".format(results_dir, configs[config]["Analytical"]))
    a_int = a_int_df["Mean"].to_list()

    # Load simulated collision
    s_int_df = pd.read_csv("{}/{}/interference_errors.csv".format(results_dir, configs[config]["Simulated"]))
    s_int = s_int_df["Mean"].to_list()

    # PDR
    size_pdr = len(a_pdr);
    sum_pdr = 0;
    for i in range(size_pdr):
        sum_pdr += abs(a_pdr[i] - s_pdr[i]);
    error_pdr = sum_pdr/size_pdr;

    # HD
    size_hd = len(a_hd);
    sum_hd = 0;
    for i in range(size_hd):
        sum_hd += abs(a_hd[i] - s_hd[i]);
    error_hd = sum_hd/size_hd;

    # SENSING
    size_uns = len(a_uns);
    sum_uns = 0;
    for i in range(size_uns):
        sum_uns += abs(a_uns[i] - s_uns[i]);
    error_uns = sum_uns/size_uns;

    # PROPAGATION
    size_prop = len(a_prop);
    sum_prop = 0;
    for i in range(size_prop):
        sum_prop += abs(a_prop[i] - s_prop[i]);
    error_prop = sum_prop/size_prop;

    # COLLISION
    size_int = len(a_int);
    sum_int = 0;
    for i in range(size_int):
        sum_int += abs(a_int[i] - s_int[i]);
    error_int = sum_int/size_int;

    
    print("|{:>13}| {:.2f}| {:.2f}| {:.2f} | {:.2f} | {:.2f} |".format(config,
                                                                       error_pdr,
                                                                       error_hd,
                                                                       error_uns,
                                                                       error_prop,
                                                                       error_int
                                                                      ))
print("="*47)

|Config       | PDR |  HD |  SEN |  PRO |  COL |
|      0.06vpm| 1.17| 0.12| 0.12 | 0.26 | 1.10 |
|      0.09vpm| 1.37| 0.08| 0.09 | 0.23 | 1.18 |
| 0.12vp-10dbm| 1.09| 0.08| 0.08 | 0.48 | 0.71 |
|0.12vpm-20dbm| 1.02| 0.07| 0.05 | 0.19 | 0.84 |
|0.12vpm-23dbm| 1.42| 0.08| 0.03 | 0.07 | 1.51 |
|      0.16vpm| 1.40| 0.06| 0.04 | 0.23 | 1.29 |
|       0.2vpm| 1.72| 0.05| 0.03 | 0.13 | 1.71 |
|      0.25vpm| 1.61| 0.04| 0.03 | 0.13 | 1.56 |
|       0.3vpm| 1.08| 0.03| 0.03 | 0.07 | 1.01 |
